# 决策树

决策树是一种基本的分类与回归方法。在分类问题中，表示基于特征对实例进行分类的过程。它可以认为是 if-then 规则的集合，也可以认为是定义在特征空间与类空间上的条件概率分布。

决策树学习通常包括 3 个步骤：`特征选择`、`决策树的生成`和`决策树的修剪`。这些决策树学习的思想主要来源于由 Quinlan 在 1986 年提出的 ID3 算法和 1993 年提出的 C4.5 算法，以及由 Breiman 等人在 1984 年提出的 CART 算法。这三个算法最大的不同在于特征选择的指标上，三者分别使用：信息增益、信息增益率以及基尼指数作为特征选择指标。本文将介绍 ID3 以及 C4.5 算法，并在下一篇文章中介绍 CART 算法。

# 1. 特征选择